# Correctness Test

## Set up

In [2]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
import time


credentialsNeo4j = pd.read_json('../credentialsNeo4j.json')

use_local = False
location = "local"

uri = "neo4j://localhost:"

if use_local == True:
    location = 'local'
    port     = 7687
else:
    location = 'server'
    port     = 7688
    
username = credentialsNeo4j[location][0]['user']
password = credentialsNeo4j[location][0]['pwd']
driver = GraphDatabase.driver(uri + str(port), auth=(username, password))
session = driver.session()

In [10]:
## parameter settings

# original set
b = 'Block'
t = 'Transaction'
a = 'Address'

# test set
bt = b + '_test'
tt = t + '_test'
at = a + '_test'

# analysis parameters
blockClasses       = [b, bt]
transactionClasses = [t, tt]
addressClasses     = [a, at]
start_block_height = 1

In [4]:
##  test settings
'''height=1
txid = '7b8f901f2244a3dcc9d2757cefe660849d8800a964fda944922df06450468153'
blockClass = b
txClass = t
addrClass = a'''

"height=1\ntxid = '7b8f901f2244a3dcc9d2757cefe660849d8800a964fda944922df06450468153'\nblockClass = b\ntxClass = t\naddrClass = a"

## Analysis

### Functions

#### Queries

In [5]:
### Blocks
## properties
def getBlockPropertiesByHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}}) RETURN b'#.format(blockClass, height)

## edges
def getBlockEdgesByHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}})<-[r]-() RETURN r'


###  Transactions
## retrieve transactions
def getTransactionsFromBlockHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}})<-[:BELONGS_TO]-(t) RETURN t.txid'

## TR properties
#def getTransactionPropertiesByTxidQuery(txClass, txid):
#    return f'MATCH (t:{txClass} {{txid: \"{txid}\"}}) RETURN t'

# alternative version
def getTransactionPropertiesByTxidQuery(txClass, txid):
    return f'MATCH (t:{txClass} {{txid: \"{txid}\"}}) RETURN t.txid, t.inDegree, t.outDegree, t.inSum, t.outSum, t.date'

### Addresses
## retrieve addresses
#def getAddressFromTransactionIDQuery(txClass, txid, addrClass):
#    return f'MATCH (t:{txClass} {{txid: \"{txid}\"}})-[]-(a:{addrClass}) RETURN a'

# alternative without addressclass
def getAddressFromTransactionIDQuery(txClass, txid):
    #return f'MATCH (t:{txClass} {{txid: \"{txid}\"}}) WHERE (t)<-[:SENDS]-(a) OR (t)-[:RECEIVES]->(a) RETURN a'
    return f'''
    MATCH (t:{txClass} {{txid: \"{txid}\"}}) WITH t MATCH (a)-[:SENDS]->(t)    RETURN a.address as address
    UNION
    MATCH (t:{txClass} {{txid: \"{txid}\"}}) WITH t MATCH (a)<-[:RECEIVES]-(t) RETURN a.address as address
    '''
    
        
## properties  
def getAddressPropertiesByAddressQuery(addrClass, address):
    return f'MATCH (a:{addrClass} {{address: \"{address}\"}}) RETURN a'

# todo: match properties of edges

#### Other functions

In [6]:
def runQuery(query):
#    return session.run(query).data()
    return session.run(query)#.values()


## second version 
def matchProperties(nodeClassList, 
                    query,
                    returnType, 
                    node_identifier, 
                    match_count, 
                    mismatches, 
                    matchAddressProperties=False):
    
    outputs = []
    
    #loop through different labels of the nodes
    for nodeClass in nodeClassList:
        #send query to neo4j
        #print(query(nodeClass, node_identifier))
        results = runQuery(query(nodeClass, node_identifier))
        
        
        ## strip off data
        # data for properties of single node
        if returnType == 'data':
            results = results.data()
            
        # values for list of one property
        elif returnType == 'values':
            results = results.values()
            
        # parameter missing
        else:
            raise Exception('returnType not set')
        
        outputs.append(results)
    
    
    #print(outputs[0])
        
    if len(outputs) != 2:
        raise Exception('Length of query output does not match.')

        
    # check if equivalent: 
    ## todo: - extend to comparison of arbitrary many 
    ##       - extend for addresses (differing in coinbase and in/outdegree)

    #outputs arre the same
    if compareOutputs(outputs, returnType) == True: 
        match_count += 1

    else:
        # todo: extend outputs to dictionary containing all relevant characters
        mismatches.append(outputs)

    return match_count, mismatches, outputs


def compareOutputs(outputs, returnType):
    
    #outputs = [v for k,v in outputs]
    #print(outputs)
    match = False 
    o1 = outputs[0]
    o2 = outputs[1]
    
    if returnType == 'values':
        if np.all(np.isin(o1, o2)):
            match = True
    
    elif o1 == o2:
        match = True
        
    return match




### Correctnes check

In [9]:
# overall parameter
mismatches  = []
match_count = 0 

#timing
startTime = time.time()

#loop through blocks
for block_height in np.arange(start_block_height, 1000):
    
    # check whether returned block properties match
    match_count, mismatches, outputs = matchProperties(nodeClassList = blockClasses, 
                                                       query = getBlockPropertiesByHeightQuery,
                                                       returnType = 'data',
                                                       node_identifier = block_height,
                                                       match_count = match_count,
                                                       mismatches = mismatches)
    

    # check whether returned txids  match
    match_count, mismatches, outputs = matchProperties(nodeClassList = blockClasses, 
                                                       query = getTransactionsFromBlockHeightQuery,
                                                       returnType = 'values',
                                                       node_identifier = block_height,
                                                       match_count = match_count,
                                                       mismatches = mismatches)
     
    # strip off outputs for 
    txids = []
    for tx_list in outputs:
        for txid in tx_list:
            txids.append(txid)
    # deduplicate list. 
    # -> if one label returns a wrong txid, it will be searched for this in both labels
    txids = np.unique(txids)
    
    # start loop for all txids for that block
    for txid in txids:

        
        # Match tx properties
        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getTransactionPropertiesByTxidQuery,
                                                           returnType = 'data',
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)
        # retrieve and match addresses
        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getAddressFromTransactionIDQuery,
                                                           returnType = 'values',
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)

        
        #### Todo: loop through edges to addresses and check properties

        
endTime = np.round((time.time() - startTime),2)
print(f'Retrieval execution time in {endTime} seconds.')
print('Retrieved blocks: ', block_height - start_block_height)
print(f'\nMatches: {match_count}; Mismatches: {len(mismatches)}')
mismatches

Retrieval execution time in 819.11 seconds.

Matches: 4034; Mismatches: 0


[]

## End connection

In [ ]:
session.close()
driver.close()